In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
#Train test split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance, test_size=0.2, random_state=42)

In [8]:
X_train

,age,affordibility
17,58,1
22,40,1
11,28,1
13,29,0
15,55,1
1,25,0
4,46,1
5,56,1
2,47,1
16,25,0


In [10]:
#Preprocessing the data
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled["age"]/100

In [11]:
import tensorflow as tf
from tensorflow import keras

In [12]:
#create model
model = keras.models.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation="sigmoid",kernel_initializer="ones",bias_initializer="zeros")
])

e:\padhai ka saman\python\Deep learning\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(loss="binary_crossentropy", 
              optimizer="adam",
              metrics=['accuracy'])
model.fit(X_train_scaled,y_train,epochs=5000)

Epoch 1/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step - accuracy: 0.5000 - loss: 0.7428
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5000 - loss: 0.7424
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5000 - loss: 0.7420
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5000 - loss: 0.7416
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5000 - loss: 0.7411
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5000 - loss: 0.7407
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5000 - loss: 0.7403
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5000 - loss: 0.7399
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5000 - loss: 0.7395
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5000 - loss: 0.7390
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5000 - loss: 0.7386
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy

In [14]:
model.evaluate(X_test_scaled,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 1.0000 - loss: 0.2647


[0.26470300555229187, 1.0]

In [15]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


array([[0.81626713],
       [0.7576754 ],
       [0.8236742 ],
       [0.18819511],
       [0.38559136],
       [0.1959822 ]], dtype=float32)

In [16]:
y_test

9     1
25    1
8     1
21    0
0     0
12    0
Name: bought_insurance, dtype: int64

In [17]:
coef, intercept = model.get_weights()
coef, intercept

(array([[5.0183096],
        [1.1966311]], dtype=float32),
 array([-2.7665412], dtype=float32))

In [21]:
# Hardcoded
import math
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

#example
sigmoid(20)

np.float64(0.9999999979388463)

In [19]:
X_test_scaled

,age,affordibility
9,0.61,1
25,0.54,1
8,0.62,1
21,0.26,0
0,0.22,1
12,0.27,0


In [22]:
def prediction_function(age,affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility
    return sigmoid(weighted_sum)

#test function
prediction_function(X_test_scaled['age'],X_test_scaled['affordibility'])


9     0.986044
25    0.980286
8     0.986718
21    0.786635
0     0.908932
12    0.794937
dtype: float64

In [23]:
def log_loss(y_true,y_pred):
    epsilon = 1e-15
    y_pred_new = [max(i,epsilon) for i in y_pred]
    y_pred_new = [min(i,1-epsilon) for i in y_pred_new]
    y_pred_new = np.array(y_pred_new)
    return -np.mean(y_true*np.log(y_pred_new)+(1-y_true) * np.log(1-y_pred_new))

In [27]:
def gradient_descent(age, affordibility, y_true, epochs, loss_threshold):
    w1 = w2 = 1
    bias = 0
    learning_rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + bias
        y_predicted = sigmoid(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        #derivatives
        w1d = (1/n)* np.dot(np.transpose(age),(y_predicted-y_true))
        w2d = (1/n)* np.dot(np.transpose(affordibility),(y_predicted-y_true))

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - (learning_rate * w1d)
        w2 = w2 - (learning_rate * w2d)

        print(f'Epoch: {i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_threshold:
            break
    return w1, w2, bias


In [31]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4948)

Epoch: 0, w1:0.9736899318847281, w2:0.931388810977659, bias:0, loss:0.7428288579142563
Epoch: 1, w1:0.9492722836283222, w2:0.8671452594841366, bias:0, loss:0.7323761856384968
Epoch: 2, w1:0.9266998810518501, w2:0.8071711720521665, bias:0, loss:0.723246404423731
Epoch: 3, w1:0.9059133490480092, w2:0.7513381912982909, bias:0, loss:0.7153204802960138
Epoch: 4, w1:0.8868432158992127, w2:0.6994929143309135, bias:0, loss:0.7084785787045061
Epoch: 5, w1:0.8694121006495444, w2:0.6514622154045587, bias:0, loss:0.7026032793088015
Epoch: 6, w1:0.8535368615465577, w2:0.6070584589964174, bias:0, loss:0.6975821511658545
Epoch: 7, w1:0.8391306103663639, w2:0.5660843761459371, bias:0, loss:0.693309658476875
Epoch: 8, w1:0.8261045264581925, w2:0.5283374479371786, bias:0, loss:0.6896884283690997
Epoch: 9, w1:0.814369431728508, w2:0.4936137062799725, bias:0, loss:0.6866299539840408
Epoch: 10, w1:0.8038371110785945, w2:0.4617109179987327, bias:0, loss:0.6840548290040042
Epoch: 11, w1:0.7944213808838091, w

(np.float64(1.077946925981257), np.float64(-0.023569815930218712), 0)

In [29]:
coef, intercept

(array([[5.0183096],
        [1.1966311]], dtype=float32),
 array([-2.7665412], dtype=float32))